# 모델 평가

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def calculate_metrics(y_true, y_pred):
    rmse = calculate_rmse(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return rmse, r2, mae

def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def evaluate_model_fit(train_rmse, val_rmse, threshold=0.17):
    """
    모델의 성능을 평가하고 오버피팅/언더피팅 여부를 판단합니다.
    
    :param train_rmse: 훈련 데이터의 RMSE
    :param val_rmse: 검증 데이터의 RMSE
    :param threshold: 오버피팅/언더피팅을 판단하는 임계값 (기본값: 0.17)
    :return: 평가 결과 문자열, 상태
    """
    relative_diff = (val_rmse - train_rmse) / train_rmse

    if relative_diff > threshold:
        return f"오버피팅 징후가 있습니다. (rmse 차이: {relative_diff:.4f})", "오버피팅"
    elif relative_diff < -threshold:
        return f"언더피팅 징후가 있습니다. (rmse 차이: {relative_diff:.4f})", "언더피팅"
    else:
        return f"모델이 적절하게 피팅되었습니다. (rmse 차이: {relative_diff:.4f})", "적절"

def evaluate_set(y_true, y_pred, set_name):
    """
    단일 데이터셋에 대한 모델 평가를 수행합니다.
    """
    rmse, r2, mae = calculate_metrics(y_true, y_pred)
    
    interpretation = f"* {set_name} 세트 평가\n"
    interpretation += f"RMSE: {rmse:.4f}, R-squared: {r2:.4f}, MAE: {mae:.4f}\n"
    
    if r2 < 0.5:
        interpretation += "- R-squared 값이 낮습니다. 모델이 데이터의 변동성을 잘 설명하지 못하고 있습니다.\n"
    elif r2 < 0.7:
        interpretation += "- R-squared 값이 중간 정도입니다. 모델이 데이터의 변동성을 어느 정도 설명하고 있습니다.\n"
    else:
        interpretation += "- R-squared 값이 높습니다. 모델이 데이터의 변동성을 잘 설명하고 있습니다.\n"
    
    mae_rmse_ratio = mae / rmse
    interpretation += f"- MAE/RMSE 비율: {mae_rmse_ratio:.4f}"
    if mae_rmse_ratio > 0.9:
        interpretation += " -> MAE와 RMSE가 비슷합니다. 오차가 일관되게 분포되어 있을 가능성이 높습니다.\n"
    elif mae_rmse_ratio < 0.7:
        interpretation += " -> RMSE가 MAE보다 상당히 큽니다. 일부 큰 오차가 모델의 성능에 큰 영향을 미치고 있을 수 있습니다.\n"
    else:
        interpretation += " -> MAE와 RMSE의 차이가 적당합니다. 오차 분포가 적절해 보입니다.\n"
    
    return interpretation, rmse, r2, mae

def comprehensive_evaluation(train_result, val_result):
    train_rmse, train_r2, train_mae = train_result[1], train_result[2], train_result[3]
    val_rmse, val_r2, val_mae = val_result[1], val_result[2], val_result[3]

    comprehensive_report = "[종합 모델 평가]\n\n"
    comprehensive_report += train_result[0] + "\n"
    comprehensive_report += val_result[0] + "\n"

    # 모델 평가
    comprehensive_report += "* 모델 성능 분석\n"
    evaluation_result, fit_status = evaluate_model_fit(train_rmse, val_rmse, threshold=0.17)
    comprehensive_report += f'{evaluation_result}\n'

    # 해석 결과 출력
    if fit_status == "오버피팅":
        comprehensive_report += "제안: 모델의 복잡도를 줄이거나, 규제를 강화하거나, 더 많은 훈련 데이터를 사용해 보세요.\n"
    elif fit_status == "언더피팅":
        comprehensive_report += "제안: 모델의 복잡도를 높이거나, 더 많은 특성을 추가하거나, 특성 엔지니어링을 수행해 보세요.\n"
    else:
        comprehensive_report += "제안: 현재 모델이 잘 작동하고 있습니다. 필요하다면 미세 조정을 통해 성능을 더 개선해 볼 수 있습니다.\n"

    # 추가적인 정보 제공
    comprehensive_report += "\n* 추가 정보\n"
    comprehensive_report += f"- Train RMSE와 Validation RMSE의 절대값 차이: {abs(val_rmse - train_rmse):.4f}\n"
    comprehensive_report += f"- Train RMSE 대비 Validation RMSE의 비율: {val_rmse / train_rmse:.4f}\n"
    comprehensive_report += f"- R-squared 차이 (Train - Validation): {train_r2 - val_r2:.4f}\n"
    comprehensive_report += f"- MAE 차이 (Validation - Train): {val_mae - train_mae:.4f}\n"
    
    return comprehensive_report   